A user needs to set their userkey in order to save the data to the correct location.

This notebook can be used to:
   - add new qualitative risk assessments
   - edit all events at a qualitative level of detail
      - saving an event through this notebook may cause loss of more detailed data associated with it

Save event data with the "save threat event" buttons.  Saving the notebook will not update the threat data.

In [ ]:
userkey = "140"
dbhost  = 'localhost'

In [ ]:
import ipywidgets as widgets
import psycopg2
import qual

conn = psycopg2.connect("postgres://csc452:csc452@" + dbhost + "/csc452")  
cursor = conn.cursor()  
cursor.execute('SET mydomain.userkey = ' + userkey)


add_button = widgets.Button(description="add threat event")
def do_add(b):
    cursor.execute (qual.new_triage_stmt, ["new threat", "threat description here", 50, 50, 50, "impact name", "impact desc", "source name", "source desc", 50, 50])
    conn.commit()

add_button.on_click(do_add)

display (add_button)




In [ ]:
cursor.execute ('SELECT threat_id, source_id, impact_id, name, description, vulnerability, pervasive, impact, impact_name, impact_description, source_name, source_description, capacity, init FROM triagev')
triages = cursor.fetchall()

d = []

def update_row(x):
    print ("saving threat event " + str(x))
    rec = d[x]
    row = rec['row']
    w = rec['w']
    vuln_val = qual.qval_dict[w[5].value]
    perv_val = qual.qval_dict[w[6].value]
    imp_val = qual.qval_dict[w[7].value]
    cap_val = qual.qval_dict[w[12].value]
    init_val = qual.qval_dict[w[13].value]
    cursor.execute (qual.update_triage_stmt, [w[3].value, w[4].value, vuln_val, perv_val, imp_val, w[8].value, w[9].value, w[10].value, w[11].value, cap_val, init_val, row[0], row[1], row[2]])
    conn.commit()
    

for x in range(0, len(triages)):
    w = [0,0,0]
    w.append ( widgets.Text( value=triages[x][3], description='Threat Event:' ))
    w.append ( widgets.Textarea( value=triages[x][4], description='Description:' ))
    w.append ( widgets.Dropdown( options=qual.qscale, value=qual.qdict[qual.norm(triages[x][5])], description='Vulnerability' ))
    w.append ( widgets.Dropdown( options=qual.qscale, value=qual.qdict[qual.norm(triages[x][6])], description='Pervasiveness' ))
    w.append ( widgets.Dropdown( options=qual.qscale, value=qual.qdict[qual.norm(triages[x][7])], description='Impact' ))
    w.append ( widgets.Text( value=triages[x][8], description='Impact:' ))
    w.append ( widgets.Textarea( value=triages[x][9], description='Impact Description:' ))
    w.append ( widgets.Text( value=triages[x][10], description='Threat Source:' ))
    w.append ( widgets.Textarea( value=triages[x][11], description='Source Description:' ))
    w.append ( widgets.Dropdown( options=qual.qscale, value=qual.qdict[qual.norm(triages[x][12])], description='Threat Capacity:' ))
    w.append ( widgets.Dropdown( options=qual.qscale, value=qual.qdict[qual.norm(triages[x][13])], description='Attack Likeliness:' ))
    d.append ({ 'row' : triages[x], 'w' : w })
    b = widgets.Button (description = 'save threat event ' + str(x))
    
    
    def make_fn (n):
        w = n
        return lambda b: update_row(w)
         
    b.on_click(make_fn(x))
    
    display (widgets.VBox ([
        widgets.HBox([w[3], w[4]]),
        widgets.HBox([w[5], w[7]]),
        widgets.HBox([w[10], w[11]]),
        widgets.HBox([w[12], w[13]]),
        b,
        widgets.Label ()
    ]))

